# Format Dataset

Code below is from https://www.kaggle.com/code/eitharyassin/fake-news-detection-on-pheme-newfeatures  
Will improve/clean up later, but an alright data preparation for now

In [6]:
import numpy as np 
import pandas as pd 
import os
import json
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import tensorflow as tf
tfk = tf.keras
tfkl = tf.keras.layers

In [7]:
from keras.models import Sequential,Model
from keras.layers import Dense, LSTM, GRU, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from sklearn import metrics
from keras.initializers import Constant


from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix



import nltk # NLP  
from nltk.corpus import stopwords  
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer 
import re  

In [8]:
%cd PHEME_veracity\all-rnr-annotated-threads

c:\Users\willc\OneDrive\Documents\uni 2023\GENG5511 MPE Project\wills ensemble\PHEME_veracity\all-rnr-annotated-threads


In [9]:
# seed initialization
seed = 42

np.random.seed(seed)
tf.random.set_seed(seed)

In [10]:
# set the folders to iterate on
folds= ['charliehebdo-all-rnr-threads','ottawashooting-all-rnr-threads',
'ebola-essien-all-rnr-threads','prince-toronto-all-rnr-threads',
'ferguson-all-rnr-threads',	'putinmissing-all-rnr-threads',
'germanwings-crash-all-rnr-threads',
'gurlitt-all-rnr-threads','sydneysiege-all-rnr-threads']


texts = []
fav_counts = []
retweet_counts = []
date = []

username = []
account_date = []
protected = []
verified = []
followers = []
followings = []
tweets_count = []

hashtag = []
url = []

events = []
y = []

In [11]:
# the annotations on PHEME have 2 boolean values, 'misinformation' value and 'true' value, we want to convert them in a single label with values
# 'true', 'false' or 'unverified'. This values can be string or numerical, depending on the 'string' parameter of the function
def convert_annotations_data(annotation, string = True):
    if 'misinformation' in annotation.keys() and 'true'in annotation.keys():
        if int(annotation['misinformation'])==0 and int(annotation['true'])==0:
            if string:
                label = "unverified"
            else:
                label = 2
        elif int(annotation['misinformation'])==0 and int(annotation['true'])==1 :
            if string:
                label = "true"
            else:
                label = 1
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==0 :
            if string:
                label = "false"
            else:
                label = 0
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==1:
            print ("OMG! They both are 1!")
            print(annotation['misinformation'])
            print(annotation['true'])
            label = None
            
    elif 'misinformation' in annotation.keys() and 'true' not in annotation.keys():
        # all instances have misinfo label but don't have true label
        if int(annotation['misinformation'])==0:
            if string:
                label = "unverified"
            else:
                label = 2
        elif int(annotation['misinformation'])==1:
            if string:
                label = "false"
            else:
                label = 0
                
    elif 'true' in annotation.keys() and 'misinformation' not in annotation.keys():
        print ('Has true not misinformation')
        label = None
    else:
        print('No annotations')
        label = None
           
    return label

In [12]:
# all interesting features
for f in folds:
  path1 = os.path.join(f, 'rumours')
  for dir1 in os.listdir(path1):
        if '_' not in dir1:
          path_target  = os.path.join(path1,dir1,'annotation.json')
          file = open(path_target)
          data = json.load(file)
          target = convert_annotations_data(data)
          y.append(target)
          path2 = os.path.join(path1, dir1,'source-tweets')
          for dir2 in os.listdir(path2):
            if '_' not in dir2:
              path3  = os.path.join(path2,dir2)
              file = open(path3)
              data = json.load(file)
            
              #tweet features
              text = data['text']
              tweet_date = data['created_at']
              fav = data['favorite_count']
              retw = data['retweet_count']
                
              #user features
              usernames = data['user']['screen_name']
              account_creation = data['user']['created_at']
              is_protected = data['user']['protected']
              is_verified = data['user']['verified']
              no_followers = data['user']['followers_count']
              no_followings = data['user']['friends_count']
              no_tweets = data['user']['statuses_count']
                
              #entities
              no_hashtags = len(data['entities']['hashtags'])      
              has_url = data['entities']['urls']      
   

              texts.append(text)
              date.append(tweet_date)
              fav_counts.append(fav)
              retweet_counts.append(retw)
                                     
              username.append(usernames)
              account_date.append(account_creation)
              protected.append(is_protected)
              verified.append(is_verified)
              followers.append(no_followers)
              followings.append(no_followings)
              tweets_count.append(no_tweets)
            
              
              hashtag.append(no_hashtags)
              url.append(has_url)
            
              events.append(f)

In [13]:
df = pd.DataFrame([texts,date,fav_counts,retweet_counts,username,account_date,followers,followings,tweets_count,protected,verified,hashtag,url,events,y],['text','date','fav_count','retweet_count','username','account_date','followers','followings','tweet_count','protected','verified','no_hashtags','urls','event','target']).transpose()
df = df.infer_objects()
df.target[9]

'unverified'

In [14]:
#df.describe(include = 'all')

In [15]:
# download nltk stuff
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\willc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
# Clean text

def remove_punctuations(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_characters(text):
    return re.sub('[^a-zA-Z]', ' ', text)

def remove_url(text):
    return re.sub(r'http\S+', '', text)

import contractions
def remove_contractions(text):
    return ' '.join([contractions.fix(word) for word in text.split()])

stop_words = stopwords.words('english')
def remove_stopwords(text):
    return ' '.join([word for word in nltk.word_tokenize(text) if word not in stop_words])
# str(text).split()

#defining the object for Lemmatization
lemmatizer = WordNetLemmatizer()
#defining a function for lemming
def lemmatize_words(text):
    return ' '.join(lemmatizer.lemmatize(word) for word in text.split())

#defining the object for stemming
stemmer = PorterStemmer()
#defining a function for stemming
def stemming_words(text):
    return ' '.join(stemmer.stem(word) for word in text.split())

def clean_text(text):
    text = remove_url(text)
    text = remove_contractions(text)
    text = text.lower()
    text = remove_punctuations(text)
    text = remove_characters(text)
    text = remove_stopwords(text)
    #text = stemming_words(text)
    #text = lemmatize_words(text)
    return text
    
# apply
df['text'] = df['text'].apply(clean_text)

# Run ML algorithms

In [17]:
# Prep data / Encode text

# Remove unverified
df = df.drop(df[df.target == 'unverified'].index)

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)
X_tf = tfidf.fit_transform(df['text'].values)

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_cv = cv.fit_transform(df['text'].values)

In [18]:
# Select encoding
X = X_tf
#X = X_cv
y = df['target']

rs = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = rs)

# Set no. of folds
num_folds = 10

###Things to do: add more features

In [19]:
# SVM
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Tune hyperparameters
param_grid = {'C': [10**x for x in range(-1, 8, 1)], 
              'gamma': [10**x for x in range(1, -8, -1)]} 
# Set verbose = 3 here for more info              
grid = GridSearchCV(SVC(random_state = rs), param_grid, refit=True, cv=num_folds, verbose = 3)
grid.fit(X_train, y_train)
print(f"The best parameters for SVM are: {grid.best_params_}")

# Run model with best results
svm = grid.best_estimator_
svm.fit(X_train, y_train)
pred_y_svm = svm.predict(X_test)
acc_svm = accuracy_score(y_test, pred_y_svm)
print("Accuracy:", float("{0:.2f}".format(acc_svm*100)), "%")
cm_svm = confusion_matrix(y_test, pred_y_svm)
print(cm_svm)

Fitting 10 folds for each of 81 candidates, totalling 810 fits
[CV 1/10] END ..................C=0.1, gamma=10;, score=0.617 total time=   0.0s
[CV 2/10] END ..................C=0.1, gamma=10;, score=0.617 total time=   0.0s
[CV 3/10] END ..................C=0.1, gamma=10;, score=0.617 total time=   0.0s
[CV 4/10] END ..................C=0.1, gamma=10;, score=0.609 total time=   0.0s
[CV 5/10] END ..................C=0.1, gamma=10;, score=0.609 total time=   0.0s
[CV 6/10] END ..................C=0.1, gamma=10;, score=0.609 total time=   0.0s
[CV 7/10] END ..................C=0.1, gamma=10;, score=0.609 total time=   0.0s
[CV 8/10] END ..................C=0.1, gamma=10;, score=0.609 total time=   0.0s
[CV 9/10] END ..................C=0.1, gamma=10;, score=0.614 total time=   0.0s
[CV 10/10] END .................C=0.1, gamma=10;, score=0.614 total time=   0.0s
[CV 1/10] END ...................C=0.1, gamma=1;, score=0.672 total time=   0.0s
[CV 2/10] END ...................C=0.1, gamma=

In [20]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

# Tune hyperparameters
param_grid = {'n_neighbors': np.arange(2, 15, 1)} 
# Set verbose = 3 here for more info  
grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, cv=num_folds, verbose=3)
grid.fit(X_train, y_train)
print(f"The best parameters for KNN are: {grid.best_params_}")

# Run model
knn = grid.best_estimator_
knn.fit(X_train, y_train)
pred_y_knn = knn.predict(X_test)
acc_knn = accuracy_score(y_test, pred_y_svm)
print("Accuracy:", float("{0:.2f}".format(acc_svm*100)), "%")
cm_knn = confusion_matrix(y_test, pred_y_knn)
print(cm_knn)

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV 1/10] END ....................n_neighbors=2;, score=0.797 total time=   0.1s
[CV 2/10] END ....................n_neighbors=2;, score=0.883 total time=   0.0s
[CV 3/10] END ....................n_neighbors=2;, score=0.914 total time=   0.0s
[CV 4/10] END ....................n_neighbors=2;, score=0.883 total time=   0.0s
[CV 5/10] END ....................n_neighbors=2;, score=0.867 total time=   0.0s
[CV 6/10] END ....................n_neighbors=2;, score=0.859 total time=   0.0s
[CV 7/10] END ....................n_neighbors=2;, score=0.883 total time=   0.0s
[CV 8/10] END ....................n_neighbors=2;, score=0.906 total time=   0.0s
[CV 9/10] END ....................n_neighbors=2;, score=0.882 total time=   0.0s
[CV 10/10] END ...................n_neighbors=2;, score=0.874 total time=   0.0s
[CV 1/10] END ....................n_neighbors=3;, score=0.844 total time=   0.0s
[CV 2/10] END ....................n_neighbors=

In [21]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

# Tune hyperparameters
param_grid = {'C': [10**x for x in range(-1, 8, 1)]} 
# Set verbose = 3 here for more info  
grid = GridSearchCV(LogisticRegression(solver = 'lbfgs', max_iter=10000), param_grid, refit=True, cv=num_folds, verbose=3)
grid.fit(X_train, y_train)
print(f"The best parameters for Logistic Regression are: {grid.best_params_}")

lr = grid.best_estimator_
lr.fit(X_train, y_train)
pred_y_lr = lr.predict(X_test)
acc_lr = accuracy_score(y_test, pred_y_lr)
print("Accuracy:", float("{0:.2f}".format(acc_lr*100)), "%")
cm_lr = confusion_matrix(y_test, pred_y_lr)
print(cm_lr)

Fitting 10 folds for each of 9 candidates, totalling 90 fits
[CV 1/10] END ............................C=0.1;, score=0.727 total time=   0.0s
[CV 2/10] END ............................C=0.1;, score=0.766 total time=   0.0s
[CV 3/10] END ............................C=0.1;, score=0.734 total time=   0.0s
[CV 4/10] END ............................C=0.1;, score=0.727 total time=   0.0s
[CV 5/10] END ............................C=0.1;, score=0.766 total time=   0.0s
[CV 6/10] END ............................C=0.1;, score=0.758 total time=   0.0s
[CV 7/10] END ............................C=0.1;, score=0.773 total time=   0.0s
[CV 8/10] END ............................C=0.1;, score=0.742 total time=   0.0s
[CV 9/10] END ............................C=0.1;, score=0.748 total time=   0.0s
[CV 10/10] END ...........................C=0.1;, score=0.740 total time=   0.0s
[CV 1/10] END ..............................C=1;, score=0.852 total time=   0.0s
[CV 2/10] END ..............................C=1;

In [22]:
# AdaBoost
from sklearn.ensemble import AdaBoostClassifier

# Tune hyperparameters
param_grid = {'learning_rate': [10**x for x in range(1,-6, -1)],
              'n_estimators': [10, 50, 100, 250, 500]} 
# Set verbose = 3 here for more info  
grid = GridSearchCV(AdaBoostClassifier(n_estimators = 200, random_state = rs), param_grid, refit=True, cv=num_folds, verbose=3)
grid.fit(X_train, y_train)
print(f"The best parameters for AdaBoost are: {grid.best_params_}")

# Run model
ada = grid.best_estimator_
ada.fit(X_train, y_train)
pred_y_ada = ada.predict(X_test)
acc_ada = accuracy_score(y_test, pred_y_ada)
print("Accuracy:", float("{0:.2f}".format(acc_ada*100)), "%")
cm_ada = confusion_matrix(y_test, pred_y_ada)
print(cm_ada)

Fitting 10 folds for each of 35 candidates, totalling 350 fits
[CV 1/10] END learning_rate=10, n_estimators=10;, score=0.461 total time=   0.0s
[CV 2/10] END learning_rate=10, n_estimators=10;, score=0.477 total time=   0.0s
[CV 3/10] END learning_rate=10, n_estimators=10;, score=0.445 total time=   0.0s
[CV 4/10] END learning_rate=10, n_estimators=10;, score=0.453 total time=   0.0s
[CV 5/10] END learning_rate=10, n_estimators=10;, score=0.445 total time=   0.0s
[CV 6/10] END learning_rate=10, n_estimators=10;, score=0.484 total time=   0.0s
[CV 7/10] END learning_rate=10, n_estimators=10;, score=0.492 total time=   0.0s
[CV 8/10] END learning_rate=10, n_estimators=10;, score=0.461 total time=   0.0s
[CV 9/10] END learning_rate=10, n_estimators=10;, score=0.465 total time=   0.0s
[CV 10/10] END learning_rate=10, n_estimators=10;, score=0.465 total time=   0.0s
[CV 1/10] END learning_rate=10, n_estimators=50;, score=0.727 total time=   0.3s
[CV 2/10] END learning_rate=10, n_estimators=